In [1]:
import logging
from random import randint
import random
import errno
import functools
from loguru import logger
import signal
import time
import json
import faiss
import subprocess
import shutil
import psutil
import pickle
import heapq
from collections import defaultdict
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

import lean_dojo
from lean_dojo import *
from utils.lean_math_utils import *

In [2]:
import os
import pickle
from lean_dojo import *
import random

print("Imports done")

github_token = os.getenv("GITHUB_ACCESS_TOKEN", None)
# print("token:", github_token)
repo = LeanGitRepo(
    "https://github.com/leanprover-community/mathematics_in_lean",
    "210802794074c003ddf237ee56a0bf1fb5a9fe10"
#     "https://github.com/yangky11/miniF2F-lean4",
#     "6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27"
)

Imports done


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
import os

class InstructionResponseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instruction, response = self.data[idx]
        
        input_encoding = self.tokenizer(instruction, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        output_encoding = self.tokenizer(response, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        input_mask = input_encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_mask,
            "labels": output_encoding["input_ids"].squeeze()
        }

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Initialize tokenizer and model
# model_name = "google/byt5-small"  # You can change this to any other Seq2Seq model
model_name_atp = ""
model_name_provability = ""

model_atp = torch.load(model_name_atp)
model_provability = torch.load(model_name_provability)

# Set up optimizer
optimizer = AdamW(model_atp.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_atp.to(device)
model_provability.to(device)

model_atp.eval()
model_provability.eval()

In [4]:
tokenizer_name = "morph-labs/morph-prover-v0-7b"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.padding_side = "right"

sep_token = "<sep>"
pad_token = "<pad>"
eos_token = "<end>"

if sep_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([sep_token])
if pad_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([pad_token])

tokenizer.sep_token = sep_token
tokenizer.pad_token = pad_token

tokenizer.add_special_tokens({
    'sep_token': sep_token,
    'pad_token': pad_token,
    'eos_token': eos_token
})

3

In [ ]:
# Test the model
# random.seed(5)
def generate_response(instruction, model, return_probs = False):
    input_ids = tokenizer(instruction, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, num_beams=10, length_penalty=0, do_sample = False, return_dict_in_generate = True, num_return_sequences = 10, eos_token_id = tokenizer.eos_token_id, output_scores = True)
    seqs = [tokenizer.decode(tac, skip_special_tokens=False) for tac in output.sequences]
#     print(output.scores)
    if not return_probs:
        return seqs
    else:
        log_probs = []
        for i, sequence in enumerate(output.sequences):
            log_prob = 0
            
            for step, logits in enumerate(output.scores):
                probabilities = torch.softmax(logits[i], dim=-1)
                token_id = sequence[step + 1]  # Skip initial input tokens
                token_log_prob = torch.log(probabilities[token_id]).item()
                log_prob += token_log_prob
            log_probs.append(-log_prob)
        return seqs, log_probs

def generate_tac(instruction, model):
    res = generate_response(instruction, model)
    return res[0].split("<s> ")[1].split("<pad>")[0]

test_instruction = """R : Type u_1\ninst✝ : Ring R\na : R\n⊢ a + 0 = a"""

response = generate_response(test_instruction, model_atp, return_probs = True)
print(f"Instruction: {test_instruction}")
print(f"Response 1: {response}")

In [7]:
def remove_padding(s):
    return s.split("<s>")[1].split("<pad>")[0][1:]

In [8]:
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")

# Register the signal function handler
signal.signal(signal.SIGALRM, timeout_handler)

<Handlers.SIG_DFL: 0>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score

# Load the data
data = pd.read_csv("")

# Separate features (X) and target (y)
X = data.drop(columns=["Proven"])
y = data["Proven"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the random forest classifier
rf_model = HistGradientBoostingClassifier(max_iter = 1000)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
predictions = (predictions >= 0.5).astype(int)
print(predictions)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [10]:
# rf_model.predict([30,0.9,0.1,0,0,0,1])

In [11]:
beam_width = 10

#1.1: 1 thread 162
#1.2: 2 threads, weighted provability
#2.1: 2 threads, not beam search 192
#2.1.1: 2 threads, tacs sorted by log prob
#2.2: 4 threads, full beam search

def get_proof(file_path, full_name):
    num_attempts = 0
    while num_attempts < 3:
        try:
            theorem = Theorem(repo, file_path, full_name)
            dojo, state_0 = Dojo(theorem).__enter__()
            break
        except:
            num_attempts += 1
            pass
    if num_attempts == 3:
        print("Theorem failed to load")
        raise
    curr_state = state_0
    pp_to_state = {}
    pp_to_state[curr_state.pp] = curr_state
    
    seen = set()
    seen.add(curr_state.pp)
    
    beam = [(0, curr_state.pp, [])]
    start_time = time.time()
    states_generated = []
    num_fails = 0
    state_data = []
    
    signal.alarm(120)
    num_ops = 0
    
    while beam:
        new_beam = []

        for _, state, proof in beam[:beam_width]:
            seen.add(state)
            state = pp_to_state[state]
            states_generated.append(time.time() - start_time)

            candidates, log_probs = generate_response(state.pp, model_atp, return_probs = True)

            state_info = [state.pp]
            for i in range(len(candidates)):
                tac = candidates[i]
                prob = log_probs[i]

                if time.time() - start_time > 120:
                    state_info.append(["Out of time"])
                    state_data.append(state_info)
                    return False, state_data, [], num_ops
                tac = remove_padding(tac)

                try:
                    num_ops += 1
                    new_state = dojo.run_tac(state, tac)
                except:
                    continue

                if type(new_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
                    state_info.append(["Error", prob])
                    print("Error\n")
                    continue
                elif type(new_state) == lean_dojo.interaction.dojo.ProofFinished:
                    state_info.append(["Successful", prob, time.time() - start_time])
                    state_data.append(state_info)
                    return True, state_data, proof + [tac], num_ops

                if new_state.pp in seen:
                    continue

                pp_to_state[new_state.pp] = new_state

                provabilities = get_provability(new_state.pp, return_probs = True)
                complexity = rf_model.predict_proba([[prob] + provabilities + [i]])[0][0]
                state_info.append([new_state.pp, tac, prob, complexity])

                new_beam.append((complexity, new_state.pp, proof + [tac]))
        
        state_data.append(state_info)
        if not new_beam:
            return False, state_data, [], num_ops

        beam = heapq.nsmallest(beam_width, new_beam)

        if len(beam) == 0:
            print("Out of beams")
            return False, state_data, [], num_ops
    return False, state_data, [], num_ops

def get_provability(state, return_probs = False):
    input_ids = tokenizer(state, return_tensors="pt").input_ids.to(device)
    output = model_provability.generate(input_ids, num_beams=4, length_penalty=0, do_sample = False, return_dict_in_generate = True, eos_token_id = tokenizer.eos_token_id, output_scores = True)
    seqs = [tokenizer.decode(tac, skip_special_tokens=False) for tac in output.sequences]

    if not return_probs:
        return seqs
    else:
        probs = []
        log_probs = output.scores[3][0]

        for i in [28740, 28750, 28770, 28781, 28782]:
            probs.append(-log_probs[i].item())

        probs = list(torch.softmax(torch.tensor(probs), -1))
        probs_2 = []
        for prob in probs:
            probs_2.append(prob.item())
        return probs_2

In [12]:
def get_thms_minif2f(file_path):
    theorems_list = []
    f = open(file_path, "r")
    all_thms = f.read().split("theorem ")[1:]
    real_path = "/".join(file_path.split("/")[7:])
    for thm in all_thms:
        theorems_list.append((real_path, thm.split(" ")[0].replace("\n", "")))
    return theorems_list

In [13]:
# theorems_list = get_thms_minif2f("") + get_thms_minif2f("")
# theorems_list

In [14]:
theorems_list = [("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_neg_cancel_right"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_left_cancel"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_right_cancel"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.zero_mul"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_eq_of_add_eq_zero"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.eq_neg_of_add_eq_zero"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_zero"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_neg"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.self_sub"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.one_add_one_eq_two"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.two_mul"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_right_inv"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_one"),("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_inv_rev"),("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "absorb1"),("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "absorb2"),("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "aux1"),("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "aux2"),("MIL/C02_Basics/S03_Using_Theorems_and_Lemmas.lean", "fact1"),("MIL/C02_Basics/S03_Using_Theorems_and_Lemmas.lean", "fact2"),("MIL/C02_Basics/S04_More_on_Order_and_Divisibility.lean", "C02S04.aux"),("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.le_abs_self"),("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.neg_le_abs_self"),("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.abs_add"),("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.lt_abs"),("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.abs_lt"),("MIL/C03_Logic/S04_Conjunction_and_Iff.lean", "C03S04.aux"),("MIL/C03_Logic/S04_Conjunction_and_Iff.lean", "C03S04.not_monotone_iff"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.my_lemma4"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnUb"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnLb"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnEven"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnOdd"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.Subset.trans"),("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.SetUb"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.ConvergesTo"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_const"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_add"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_mul_const"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.exists_abs_le_of_convergesTo"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.aux"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_mul"),("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_unique"),("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnUb"),("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnLb"),("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnHasUb"),("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnHasLb"),("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.fnUb_add"),("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnUb"),("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnLb"),("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnHasUb"),("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnHasLb"),("MIL/C04_Sets_and_Functions/S02_Functions.lean", "inverse"),("MIL/C04_Sets_and_Functions/S02_Functions.lean", "inverse_spec"),("MIL/C04_Sets_and_Functions/S02_Functions.lean", "Cantor"),("MIL/C04_Sets_and_Functions/S01_Sets.lean", "primes"),("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbAux"),("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbSet"),("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbFun"),("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_right_inv"),("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_injective"),("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_surjective"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "fac"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "pow_two_le_fac"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "sum_sqr"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.zero_add"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.succ_add"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add_comm"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add_assoc"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul_add"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.zero_mul"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.succ_mul"),("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul_comm"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.two_le"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.exists_prime_factor"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_infinite"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "Nat.Prime.eq_of_dvd_of_prime"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.mem_of_dvd_prod_primes"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_infinite'"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.bounded_of_ex_finset"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.ex_finset_of_bounded"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.mod_4_eq_3_or_mod_4_eq_3"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.two_le_of_mod_4_eq_3"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.aux"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.exists_prime_factor_mod_4_eq_3"),("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_mod_4_eq_3_infinite"),("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "even_of_even_sqr"),("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "factorization_mul'"),("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "factorization_pow'"),("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "Nat.Prime.factorization'"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.instCommRing"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.sub_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.sub_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.div'"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.mod'"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.div'_add_mod'"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.abs_mod'_le"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.mod'_eq"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "aux"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "sq_add_sq_eq_zero"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_nonneg"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_eq_zero"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_pos"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_mul"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj_re"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj_im"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_conj"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.div_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mod_def"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_mod_lt"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.coe_natAbs_norm"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.natAbs_norm_mod_lt"),("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.not_norm_mul_left_lt_norm"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.AddGroup₁"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.add"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.neg"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.zero"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.addGroupPoint"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.AddGroup₂"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasAddAddGroup₂"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasZeroAddGroup₂"),("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasNegAddGroup₂"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.add"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.add_assoc"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.smul"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.smul_distrib"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardTwoSimplex"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardTwoSimplex.weightedAverage"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex.midpoint"),("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex.weightedAverage"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "Submonoid₁"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubMonoid₁Monoid"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubmonoidClass₁"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "Subgroup₁"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubgroupClass₁"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "Submonoid.Setoid"),("MIL/C07_Hierarchies/S03_Subobjects.lean", "QuotientMonoid.mk"),("MIL/C07_Hierarchies/S01_Basics.lean", "One₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "One₂"),("MIL/C07_Hierarchies/S01_Basics.lean", "Dia₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₂"),("MIL/C07_Hierarchies/S01_Basics.lean", "DiaOneClass₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₂"),("MIL/C07_Hierarchies/S01_Basics.lean", "Inv₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "Group₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "left_inv_eq_right_inv₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "inv_eq_of_dia"),("MIL/C07_Hierarchies/S01_Basics.lean", "dia_inv"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddSemigroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddMonoid₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "left_inv_eq_right_inv'"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommSemigroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "CommSemigroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommMonoid₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "CommMonoid₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddGroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "Group₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "inv_eq_of_mul"),("MIL/C07_Hierarchies/S01_Basics.lean", "Group₃.mul_inv"),("MIL/C07_Hierarchies/S01_Basics.lean", "mul_left_cancel₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "mul_right_cancel₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommGroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "CommGroup₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "Ring₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "LE₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "Preorder₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "PartialOrder₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "OrderedCommMonoid₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "SMul₃"),("MIL/C07_Hierarchies/S01_Basics.lean", "Module₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "selfModule"),("MIL/C07_Hierarchies/S01_Basics.lean", "nsmul₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "zsmul₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "abGrpModule"),("MIL/C07_Hierarchies/S01_Basics.lean", "AddMonoid₄"),("MIL/C07_Hierarchies/S01_Basics.lean", "mySMul"),("MIL/C07_Hierarchies/S01_Basics.lean", "LT₁"),("MIL/C07_Hierarchies/S01_Basics.lean", "PreOrder₂"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "isMonoidHom₁"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "isMonoidHom₂"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHom₁"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "AddMonoidHom₁"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "RingHom₁"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₁"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "badInst"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₂"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "map_inv_of_inv"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₃"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresHom"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresMonoidHom"),("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresHomClass"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "conjugate"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "eq_bot_iff_card"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "inf_bot_of_coprime"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "conjugate_one"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "aux_card_eq"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "iso₁"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "iso₂"),("MIL/C08_Groups_and_Rings/S01_Groups.lean", "finalIso"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_mk"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_mk'"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_inj"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "isCoprime_Inf"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_surj"),("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseIso"),("MIL/C09_Topology/S03_Topological_Spaces.lean", "aux"),("MIL/C09_Topology/S01_Filters.lean", "Tendsto₁"),("MIL/C09_Topology/S02_Metric_Spaces.lean", """cauchySeq_of_le_geometric_two'""")]# theorems_list = [('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean', 'MyRing.neg_add_cancel_left')]

In [15]:
theorems_list

[('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.add_neg_cancel_right'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.add_left_cancel'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.add_right_cancel'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.zero_mul'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.neg_eq_of_add_eq_zero'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.eq_neg_of_add_eq_zero'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.neg_zero'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.neg_neg'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.self_sub'),
 ('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean',
  'MyRing.one_add_one_eq_two')

In [21]:
total_attempts = 0
num_proven = 0

for thm_path, thm_name in theorems_list:
    
    print(thm_path, thm_name)   
    try:
        is_proven, state_info, proof, num_ops = get_proof(thm_path, thm_name)
        num_proven += is_proven
        total_attempts += 1
        print(f"{num_proven} theorems proven out of {total_attempts}")
        print(state_info)
        state_data = {"file_path": thm_path, "full_name": thm_name, "total_attempts": total_attempts, "is_proven": is_proven, "proof": proof, "num_ops": num_ops, "proof_info": state_info}
#         with open("", "a") as outfile:
#             json.dump(state_data, outfile)
#             outfile.write("\n")
    
    except Exception as e:
        print(e)
        state_data = {"file_path": thm_path, "full_name": thm_name, "total_attempts": total_attempts, "is_proven": False, "proof": [], "num_ops": -1, "proof_info": [False]}
#         with open("", "a") as outfile:
#             json.dump(state_data, outfile)
#             outfile.write("\n")
        total_attempts += 1
# print(1/0)